### Energieversorgung Vertiefung Hausübung 1: Zuverlässigkeitskennzahlen

In [9]:
import pandas as pd
from tabulate import tabulate

In [10]:
# Eingabe Matrikelnummer
MatrNr = 11907067

# Berechnung IPS
def calculation_ips(Matrikelnummer):     
    q = sum(int(digit) for digit in str(Matrikelnummer))
    return ((q * 5) % 40) * 10**-4

IPS = calculation_ips(MatrNr)

In [11]:
#Parameter Leitungslänge [km]
ltg_l = 5

# Parameter Ausfallhäufigkeit
h_ltg110 = 0.005 * ltg_l #[1/a]
h_ltg20 = 0.001 * ltg_l #[1/a]
h_T = 0.015 #[1/a]
h_TR = 0.00012 + IPS #[1/a]
h_ls = 0.000259 #[1/a]
h_SS = 0.000148 #[1/a]

# Parameter Ausfalldauer
ta_ltg110 = 80 #[h]
ta_ltg20 = 26 #[h]
ta_T = 28 #[h]
ta_TR = 72 #[h]
ta_ls=72 #[h]
ta_SS = 72 #[h]
ksT_UW = 0.25 # Korrekturschaltdauer im UW
ksT_aUW = 0.25 # Korrekturschaltdauer außerhalb vom UW

#Parameter Lasten [MW]
P_A = 5
P_GK = 5
P_B = 5
P_C = 5

In [12]:
# Berechnung der Zuverlässigkeitskennzahlen 
def calculate_kenngroessen(defizithaeufigkeit, ausfalldauer, last, laenge = 1):
    H = defizithaeufigkeit * laenge
    NV = H * ausfalldauer
    P = H * last
    E = NV * last

    return H, NV, P, E

# Berechnung für den ersten Fall: Aktuelle Situation, also der Großkunde wird nur vom UW1 mit der H-Schaltung versorgt
elemente_H_Schaltung = [
    {'name': 'LTG1', 'defizithaeufigkeit': h_ltg110, 'ausfalldauer': ta_ltg110, 'laenge': ltg_l},
    {'name': 'LTG2', 'defizithaeufigkeit': h_ltg110, 'ausfalldauer': ta_ltg110, 'laenge': ltg_l},
    {'name': 'HTR11', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    {'name': 'HTR12', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    {'name': 'HTR21', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    {'name': 'HTR22', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    {'name': 'QTR1', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    {'name': 'QTR2', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    {'name': 'HLS11', 'defizithaeufigkeit': h_ls, 'ausfalldauer': ta_ls},
    {'name': 'HLS12', 'defizithaeufigkeit': h_ls, 'ausfalldauer': ta_ls},
    {'name': 'T1', 'defizithaeufigkeit': h_T, 'ausfalldauer': ta_T},
    {'name': 'T2', 'defizithaeufigkeit': h_T, 'ausfalldauer': ta_T},
    {'name': 'HLS21', 'defizithaeufigkeit': h_ls, 'ausfalldauer': ta_ls},
    {'name': 'HLS22', 'defizithaeufigkeit': h_ls, 'ausfalldauer': ta_ls},
    {'name': 'HTR31', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    {'name': 'HTR32', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
]

# Erstellen eines pandas-dataframes
data = pd.DataFrame(columns=['Name', 'H [1/a]', 'NV [h/a]', 'P [MW/a]', 'E [MWh/a]'])

# Berechnung der Zuverlässigkeitskennzahlen und Hinzufügen zum DataFrame
for i, element in enumerate(elemente_H_Schaltung):
    H, NV, P, E = calculate_kenngroessen(element['defizithaeufigkeit'], element['ausfalldauer'], P_A, element.get('laenge', 1))
    
    # Hinzufügen der Daten zum DataFrame
    data.loc[i] = [element['name'], H, NV, P, E]

print(data)

# Beschreibung der Ausfallwirkung für die H-Schaltung in einer Tabelle
rows = []

# Zeilenweise Eintragen der Werte (Fehler oberhalb des Trafos führt zu Gesamtausfall 110kV-System, darunter nur der betroffenen Abzweige)
rows.append(["LTG1", "LT", "V1 + V2", "-"])
rows.append(["LTG2", "LT", "V1 + V2", "-"])
rows.append(["HTR11", "TR", "V1 + V2", "V1"])
rows.append(["HTR12", "TR", "V1 + V2", "V2"])
rows.append(["HTR21", "TR", "V1 + V2", "V1"])
rows.append(["HTR22", "TR", "V1 + V2", "V2"])
rows.append(["QTR1", "TR", "V1 + V2", "V1"])
rows.append(["QTR2", "TR", "V1 + V2", "V2"])
rows.append(["HLS11", "LS", "V1 + V2", "V1"])
rows.append(["HLS12", "LS", "V1 + V2", "V2"])
rows.append(["T1", "T", "V1", "V1"])
rows.append(["T2", "T", "V2", "V2"])
rows.append(["HLS21", "LS", "V1", "V1"])
rows.append(["HLS22", "LS", "V2", "V2"])
rows.append(["HTR31", "TR", "V1", "V1"])
rows.append(["HTR32", "TR", "V2", "V2"])

# DataFrame erstellen
df = pd.DataFrame(rows, columns=[
    "Ausgefallenes Element", 
    "Zuverlässigkeitskenn­größe, Typ", 
    "Ausgefallene Verbraucher - Nach Fehler", 
    "Ausgefallene Verbraucher - Nach Umschaltung"
])

# Tabelle ausgeben
print(tabulate(df, headers="keys", tablefmt="grid"))



     Name   H [1/a]   NV [h/a]  P [MW/a]  E [MWh/a]
0    LTG1  0.125000  10.000000  0.625000   50.00000
1    LTG2  0.125000  10.000000  0.625000   50.00000
2   HTR11  0.003620   0.260640  0.018100    1.30320
3   HTR12  0.003620   0.260640  0.018100    1.30320
4   HTR21  0.003620   0.260640  0.018100    1.30320
5   HTR22  0.003620   0.260640  0.018100    1.30320
6    QTR1  0.003620   0.260640  0.018100    1.30320
7    QTR2  0.003620   0.260640  0.018100    1.30320
8   HLS11  0.000259   0.018648  0.001295    0.09324
9   HLS12  0.000259   0.018648  0.001295    0.09324
10     T1  0.015000   0.420000  0.075000    2.10000
11     T2  0.015000   0.420000  0.075000    2.10000
12  HLS21  0.000259   0.018648  0.001295    0.09324
13  HLS22  0.000259   0.018648  0.001295    0.09324
14  HTR31  0.003620   0.260640  0.018100    1.30320
15  HTR32  0.003620   0.260640  0.018100    1.30320
+----+-------------------------+----------------------------------+------------------------------------------+------

In [13]:
# Berechnung für zweiten Fall: Verlängerung der Mittelspannungsstichleitung, Doppelsammelschiene-Topologie in UW B




In [14]:
# Berechnung für dritten Fall: Umbau von UW A auf Doppelsammelschienen-Topologie


elemente_H_Schaltung = [
    {'name': 'LTG1', 'defizithaeufigkeit': h_ltg110, 'ausfalldauer': ta_ltg110, 'laenge': ltg_l},
    {'name': 'LTG2', 'defizithaeufigkeit': h_ltg110, 'ausfalldauer': ta_ltg110, 'laenge': ltg_l},
    
    {'name': 'LTR11', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    {'name': 'LTR12', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    
    {'name': 'LS11', 'defizithaeufigkeit': h_ls, 'ausfalldauer': ta_ls},
    {'name': 'LS12', 'defizithaeufigkeit': h_ls, 'ausfalldauer': ta_ls},
    
    {'name': 'STR11', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    {'name': 'STR12', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    
    {'name': 'STR21', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    {'name': 'STR22', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    
    {'name': 'QTR1', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    {'name': 'QTR2', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    {'name': 'QLS', 'defizithaeufigkeit': h_ls, 'ausfalldauer': ta_ls},
    
    {'name': 'SS1', 'defizithaeufigkeit': h_SS, 'ausfalldauer': ta_SS},
    {'name': 'SS2', 'defizithaeufigkeit': h_SS, 'ausfalldauer': ta_SS},
    
    {'name': 'STR31', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    {'name': 'STR32', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    
    {'name': 'STR41', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    {'name': 'STR42', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    
    {'name': 'LS21', 'defizithaeufigkeit': h_ls, 'ausfalldauer': ta_ls},
    {'name': 'LS22', 'defizithaeufigkeit': h_ls, 'ausfalldauer': ta_ls},
    
    {'name': 'T1', 'defizithaeufigkeit': h_T, 'ausfalldauer': ta_T},
    {'name': 'T2', 'defizithaeufigkeit': h_T, 'ausfalldauer': ta_T},
    
    {'name': 'LS31', 'defizithaeufigkeit': h_ls, 'ausfalldauer': ta_ls},
    {'name': 'LS32', 'defizithaeufigkeit': h_ls, 'ausfalldauer': ta_ls},
    
    {'name': 'ATR11', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
    {'name': 'ATR12', 'defizithaeufigkeit': h_TR, 'ausfalldauer': ta_TR},
]

# Erstellen eines pandas-dataframes
data = pd.DataFrame(columns=['Name', 'H [1/a]', 'NV [h/a]', 'P [MW/a]', 'E [MWh/a]'])

# Berechnung der Zuverlässigkeitskennzahlen und Hinzufügen zum DataFrame
for i, element in enumerate(elemente_H_Schaltung):
    H, NV, P, E = calculate_kenngroessen(element['defizithaeufigkeit'], element['ausfalldauer'], P_A, element.get('laenge', 1))
    
    # Hinzufügen der Daten zum DataFrame
    data.loc[i] = [element['name'], H, NV, P, E]

print(data)

# Beschreibung der Ausfallwirkung für die Doppelsammelschienen-Schaltanlage in einer Tabelle
rows = []

# Der in der Abbildung ersichtliche Schaltzustand wird als Basis für nachfolgende Überlegungen getroffen

# Zeilenweise Eintragen der Werte
rows.append(["LTG1", "LT", "-", "-"])
rows.append(["LTG2", "LT", "-", "-"])
rows.append(["LTR11", "TR", "-", "-"])
rows.append(["LTR12", "TR", "-", "-"])
rows.append(["LS11", "LS", "V1", "-"]) # Annahme Fehlerbereich freigeschaltet (Kupplung, LTG), dann Trenner geöffnet + Kupplung zu → Wiederversorgung V1
rows.append(["LS12", "LS", "V2", "-"])


rows.append(["STR11", "TR", "V1", "-"]) # Ein hier benötigter Sammelschienenwechsel (Wiederversorgung V1) benötigt entweder das gleiche Potential oder eine Spannungsfreiheit, die nur durch Schalten von LS12 erreicht wird und damit den Ausfall beider Kunden zur Folge hat
rows.append(["STR12", "TR", "V2", "-"]) # test

rows.append(["STR21", "TR", "-", "-"])
rows.append(["STR22", "TR", "V2", "-"])

rows.append(["QTR1", "TR", "V1 + V2", "-"]) # sind uns bei der Querkupplung nicht sicher!
rows.append(["QTR2", "TR", "V1 + V2", "-"])
rows.append(["QLS", "LS", "V1 + V2", "-"])

rows.append(["SS1", "SS", "V1", "-"])
rows.append(["SS2", "SS", "V2", "-"])

rows.append(["STR31", "TR", "V1", "-"])
rows.append(["STR32", "TR", "-", "-"])

rows.append(["STR41", "TR", "-", "-"])
rows.append(["STR42", "TR", "V2", "-"])

rows.append(["LS21", "LS", "V1", "V1"])
rows.append(["LS22", "LS", "V2", "V2"])

rows.append(["T1", "T", "V1", "V1"])
rows.append(["T2", "T", "V2", "V2"])

rows.append(["LS31", "LS", "V1", "V1"])
rows.append(["LS32", "LS", "V2", "V2"])

rows.append(["ATR11", "TR", "V1", "V1"])
rows.append(["ATR12", "TR", "V2", "V2"])

# DataFrame erstellen
df = pd.DataFrame(rows, columns=[
    "Ausgefallenes Element", 
    "Zuverlässigkeitskenn­größe, Typ", 
    "Ausgefallene Verbraucher - Nach Fehler", 
    "Ausgefallene Verbraucher - Nach Umschaltung"
])

# Tabelle ausgeben
print(tabulate(df, headers="keys", tablefmt="grid"))

     Name   H [1/a]   NV [h/a]  P [MW/a]  E [MWh/a]
0    LTG1  0.125000  10.000000  0.625000   50.00000
1    LTG2  0.125000  10.000000  0.625000   50.00000
2   LTR11  0.003620   0.260640  0.018100    1.30320
3   LTR12  0.003620   0.260640  0.018100    1.30320
4    LS11  0.000259   0.018648  0.001295    0.09324
5    LS12  0.000259   0.018648  0.001295    0.09324
6   STR11  0.003620   0.260640  0.018100    1.30320
7   STR12  0.003620   0.260640  0.018100    1.30320
8   STR21  0.003620   0.260640  0.018100    1.30320
9   STR22  0.003620   0.260640  0.018100    1.30320
10   QTR1  0.003620   0.260640  0.018100    1.30320
11   QTR2  0.003620   0.260640  0.018100    1.30320
12    QLS  0.000259   0.018648  0.001295    0.09324
13    SS1  0.000148   0.010656  0.000740    0.05328
14    SS2  0.000148   0.010656  0.000740    0.05328
15  STR31  0.003620   0.260640  0.018100    1.30320
16  STR32  0.003620   0.260640  0.018100    1.30320
17  STR41  0.003620   0.260640  0.018100    1.30320
18  STR42  0